In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

# Set up parameters

In [2]:
dataset = 'ling'
trainDataDir = '/Users/SamZhang/Documents/Capstone/dataset/' + dataset + '/train'

In [3]:
# Data loading params
tf.flags.DEFINE_float("dev_sample_percentage", .2, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", trainDataDir + "/spam/" + dataset + "_train.spam", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", trainDataDir + "/ham/" + dataset + "_train.ham", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 256, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.1, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 128, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

In [4]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=128
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.2
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.1
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/ling/train/ham/ling_train.ham
NUM_CHECKPOINTS=5
NUM_EPOCHS=10
NUM_FILTERS=256
POSITIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/ling/train/spam/ling_train.spam



# Process data

In [5]:
def processData():
    # Data Preparation
    # ==================================================

    # Load data
    print("Loading data...")
    x_text, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)
    print(x_text[0])

    # Build vocabulary
    max_document_length = max([len(x.split(" ")) for x in x_text])
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
    dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
    
    return x_train, y_train, x_dev, y_dev, vocab_processor

In [6]:
x_train, y_train, x_dev, y_dev, vocab_processor = processData()

Loading data...
great part time summer job ! display box credit application need place small owner operate store area here 1 introduce yourself store owner manager 2 our 90 effective script tell little display box save customer hundred dollar , draw card business , 5 0 15 0 every app send 3 spot counter , place box , nothing need , need name address company send commission check compensaation 10 every box place become representative earn commission 10 each application store course much profitable plan , pay month small effort call 1 888 703 5390 code 3 24 hours receive detail ! ! removed our mailing list , type b2998 hotmail com \( \) area \( remove \) subject area e mail send
Vocabulary Size: 38556
Train/Dev split: 1312/327


# Building CNN

In [7]:
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor.vocabulary_),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", "cnnmodel"))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/b:0/grad/hist is illegal; using 

2018-03-23T22:27:25.389066: step 97, loss 0.307092, acc 0.984375
2018-03-23T22:27:45.128512: step 98, loss 0.253738, acc 0.96875
2018-03-23T22:27:49.464903: step 99, loss 0.184802, acc 1
2018-03-23T22:28:07.105455: step 100, loss 0.3578, acc 0.960938

Evaluation:
2018-03-23T22:28:27.342586: step 100, loss 0.256721, acc 0.972477

Saved model checkpoint to /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/model-100

2018-03-23T22:28:45.985141: step 101, loss 0.201092, acc 0.992188
2018-03-23T22:29:03.577290: step 102, loss 0.270978, acc 0.976562
2018-03-23T22:29:21.316743: step 103, loss 0.289877, acc 0.960938
2018-03-23T22:29:40.045398: step 104, loss 0.304361, acc 0.945312
2018-03-23T22:29:58.536739: step 105, loss 0.362145, acc 0.953125
2018-03-23T22:30:17.313836: step 106, loss 0.305774, acc 0.96875
2018-03-23T22:30:36.428031: step 107, loss 0.463474, acc 0.953125
2018-03-23T22:30:54.810420: step 108, loss 0.379766, acc 0.945312
2018-03-23T22:31:14.047797: step 109,

# Testing model

In [8]:
from sklearn import metrics
import csv

In [9]:
testDataDir = "/Users/SamZhang/Documents/Capstone/dataset/" + dataset + "/test"

In [10]:
tf.flags.DEFINE_string("positive_test_data_file", testDataDir + "/spam/" + dataset + "_test.spam", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_test_data_file", testDataDir + "/ham/" + dataset + "_test.ham", "Data source for the negative data.")

# Eval Parameters
tf.flags.DEFINE_string("checkpoint_dir", "/Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/", "Checkpoint directory from training run")

In [11]:
FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=128
CHECKPOINT_DIR=/Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.2
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.1
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/ling/train/ham/ling_train.ham
NEGATIVE_TEST_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/ling/test/ham/ling_test.ham
NUM_CHECKPOINTS=5
NUM_EPOCHS=10
NUM_FILTERS=256
POSITIVE_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/ling/train/spam/ling_train.spam
POSITIVE_TEST_DATA_FILE=/Users/SamZhang/Documents/Capstone/dataset/ling/test/spam/ling_test.spam



In [12]:
x_raw, y_test = data_helpers.load_data_and_labels(FLAGS.positive_test_data_file, FLAGS.negative_test_data_file)
y_test = np.argmax(y_test, axis=1) #ham = 0, spam = 1
print(x_raw[0], y_test[0])

3500 per week home computer ! put free software computer start huge amount cash without work ! ! ! ! computer does work money ! ! ! never talk anybody ! ! load free software computer let believe much money rolling efforts computer ! ! ! information email manboca hotmail com 1


In [13]:
vocab_path = os.path.join(FLAGS.checkpoint_dir, "..", "vocab")
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
x_test = np.array(list(vocab_processor.transform(x_raw)))

In [14]:
checkpoint_file = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)

In [15]:
graph = tf.Graph()
with graph.as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        # Load the saved meta graph and restore variables
        saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
        saver.restore(sess, checkpoint_file)

        # Get the placeholders from the graph by name
        input_x = graph.get_operation_by_name("input_x").outputs[0]
        # input_y = graph.get_operation_by_name("input_y").outputs[0]
        dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

        # Tensors we want to evaluate
        predictions = graph.get_operation_by_name("output/predictions").outputs[0]

        # Generate batches for one epoch
        batches = data_helpers.batch_iter(list(x_test), FLAGS.batch_size, 1, shuffle=False)

        # Collect the predictions here
        all_predictions = []

        for x_test_batch in batches:
            batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
            all_predictions = np.concatenate([all_predictions, batch_predictions])

# Print accuracy if y_test is defined
if y_test is not None:
    correct_predictions = float(sum(all_predictions == y_test))
    print("Total number of test examples: {}".format(len(y_test)))
    print("Accuracy: {:g}".format(correct_predictions/float(len(y_test))))

INFO:tensorflow:Restoring parameters from /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/model-100
Total number of test examples: 1254
Accuracy: 0.992823


In [16]:
# Save the evaluation to a csv
title = np.column_stack(('text', 'prediction', 'label'))
predictions_human_readable = np.column_stack((np.array(x_raw), all_predictions, y_test))
out_path = os.path.join(FLAGS.checkpoint_dir, "..", "prediction.csv")
print("Saving evaluation to {0}".format(out_path))

with open(out_path, 'w') as f:
    csv.writer(f).writerows(title)
    csv.writer(f).writerows(predictions_human_readable)

Saving evaluation to /Users/SamZhang/Documents/Capstone/Models/runs/cnnmodel/checkpoints/../prediction.csv
